In [2]:
import litellm
import time
import sys, os

if not os.environ["OPENAI_API_KEY"]: 
    os.environ["OPENAI_API_KEY"]= '<REDACTED>'

In [22]:
MODEL = "ft:gpt-4o-mini-2024-07-18:personal:20qs-ft-1:9tKOfrv9"
#MODEL = "gpt-4o-mini"
EVALUATOR_MODEL="gpt-4o-mini"
# MODEL = "gpt-4o"
#MODEL = "claude-3-5-sonnet-20240620"
litellm.modify_params = True

In [4]:
def completion(model, messages):
    result = litellm.completion(model, messages)
    if "claude" in model:
        time.sleep(60/50) # Adjust based on rate limit (https://console.anthropic.com/settings/limits)
    return result

In [5]:
class DecisionNode:
    def __init__(self, query = None, yes_branch=None, no_branch=None):
        self.query = query
        self.yes_branch = yes_branch
        self.no_branch = no_branch
        
    def print_tree(self, level=0, prefix=""):
        if self.query:
            print(f"{prefix}-- {self.query}")
            if self.yes_branch:
                self.yes_branch.print_tree(level + 1, prefix + "  |")
            if self.no_branch:
                self.no_branch.print_tree(level + 1, prefix + "   ")
        else:
            print(f"{prefix}-- *")
            
    def write_tree_to_file(self, filename):
        with open(filename, 'w') as f:
            self._write_tree(f)

    def _write_tree(self, file, level=0, prefix=""):
        if self.query:
            file.write(f"{prefix}-- {self.query}\n")
            if self.yes_branch:
                self.yes_branch._write_tree(file, level + 1, prefix + "  |")
            if self.no_branch:
                self.no_branch._write_tree(file, level + 1, prefix + "   ")
        else:
            file.write(f"{prefix}-- *\n")

    @staticmethod
    def read_tree_from_file(filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
        return DecisionNode._read_tree(lines, 0, "")

    @staticmethod
    def _read_tree(lines, indent_level, current_prefix):
        if not lines:
            return None

        line = lines.pop(0).rstrip()
        expected_prefix = current_prefix + "-- "

        if not line.startswith(expected_prefix):
            return None

        content = line[len(expected_prefix):]
        if content == "*":
            return DecisionNode()

        yes_branch = DecisionNode._read_tree(lines, indent_level + 1, current_prefix + "  |")
        no_branch = DecisionNode._read_tree(lines, indent_level + 1, current_prefix + "   ")

        return DecisionNode(content, yes_branch, no_branch)

# Creating the tree

In [6]:
things_tree = DecisionNode.read_tree_from_file('20qs-data/decision_trees/decision_tree_v2.txt')
things_tree.print_tree()

-- food, beverage, cooking
  |-- food
  |  |-- agriculture
  |  |  |-- *
  |  |   -- prepared food
  |  |     |-- *
  |  |      -- *
  |   -- agriculture
  |     |-- safety
  |     |  |-- *
  |     |   -- *
  |      -- handheld
  |        |-- beverage
  |        |  |-- *
  |        |   -- *
  |         -- *
   -- industry, manufacturing
     |-- safety
     |  |-- transportation, vehicles
     |  |  |-- *
     |  |   -- architecture, construction
     |   -- electronics
     |     |-- *
     |      -- natural material, resource
     |        |-- *
     |         -- *
      -- sports, entertainment
        |-- handheld
        |  |-- electronics
        |  |  |-- *
        |  |   -- arts, media
        |  |     |-- *
        |  |      -- *
        |   -- *
         -- clothing, accessories, beauty
           |-- *
            -- animal
              |-- *
               -- handheld
                 |-- electronics
                 |  |-- *
                 |   -- safety
                

In [7]:
query_map = {
    'food': "Is it a food?",
    'beverage': "Is it a beverage?",
    'living': "Is it a living thing?",
    'plant': "Is it a plant?",
    'animal': 'Is it an animal?',
    'electronics': "Is it related to electronics or technology?",
    'furniture': 'Is it furniture?',
    'transportation, vehicles': 'Is it related to transportation or vehicles?',
    'man-made': "Is it a man-made thing?",
    'architecture, construction': 'Is it related to architectural structures or construction?',
    'natural material, resource': 'Is it a natural material or resource?',
    'natural phenomenon': 'Is it a natural phenomenon?',
    'industry, manufacturing': "Is it related to industrial production or manufacturing?",
    'food, beverage, cooking': 'Is it related to food, beverages or cooking?',
    'handheld': 'Is it something a person can hold in their hand?',
    'agriculture': "Is it related to agriculture?",
    'indoors': 'Is it something that can be found indoors?',
    'arts, media': "Is it broadly related to the arts or media?",
    'safety': "Is it related to safety or safety equipment?",
    'medicine': "Is it broadly related to medicine or healthcare?",
    'clothing, accessories, beauty': "Is it related to clothing, accessories or beauty products?",
    'sports, entertainment': "Is it broadly related to entertainment or sports?",
    'prepared food': "Is it a prepared food or dish?"
}


In [8]:
def game_to_string(game):
    return '\n'.join([f'{step["role"]}: {step["content"]}' for step in game])

def construct_messages(game, role, keyword=None, guesses=[]):
    if role == "questioner":
        prompt = {
            "role": "system",
            "content": (
                "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "
                "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                "Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person.\n\n "
                "You are participating in a new game of 20 Questions. Your role is to be the Questioner. You will ask successive yes-or-no questions to determine the keyword. "
                "You have a limited number of questions to ask, so choose a question that will eliminate half of the possible keywords to maximize efficiency. "
                "Avoid asking questions that are too specific too early on. Be as vague as possible while still eliminating half of the remaining possibilities. "
                "DO NOT ask if the keyword is a specific thing, rather ask something about the keyword.\n"
                "Example 1: DO NOT ASK: 'Is the keyword car?', INSTEAD ASK: 'Is it a specific type of car, such as a sedan?\n"
                "Example 2: DO NOT ASK: 'Is the keyword cow?', INSTEAD ASK: 'Is it a specific type of cow?\n"
                "Example 3: DO NOT ASK: 'Is the keyword bottle?', INSTEAD ASK: 'Is it a bottle made of a specific material?\n"
                "Example 4: DO NOT ASK: 'Is the keyword lamp?', INSTEAD ASK: 'Is it a type of lamp?\n"
                "Do NOT assume the game has ended, the game will determine when to stop. Do not output any text other than the question."
            ),
        }
        messages = [prompt]
        for message in game:
            if message["role"] == "questioner":
                messages.append({
                    "role": "assistant",
                    "content": message["content"]
                })
            else:
                messages.append({
                    "role": "user",
                    "content": message["content"]
                })
    elif role == "answerer":
        prompt = {
            "role": "system",
            "content": (
                f"You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "
                "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                "Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person. "
                "You are participating in a new game of 20 Questions. Your role is to be the Answerer. "
                f"The keyword is {keyword}. Answer only Yes or No based on the keyword. Do not output any other text."
            ),
        }
        messages = [prompt]
        for message in game:
            if message["role"] == "questioner":
                messages.append({
                    "role": "user",
                    "content": f"Answer the following question about the keyword: '{keyword}'. " + message["content"]
                })
            else:
                messages.append({
                    "role": "assistant",
                    "content": message["content"]
                })
    elif role == "guesser":
        prompt = {
            "role": "system",
            "content": (
                "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "
                "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "
                "Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person. "
                "You are participating in a new game of 20 Questions. Your role is to be the Guesser. Based on the given questions and answers, guess the keyword at this point. "
                "Even if information is limited, guess a keyword. Do not ask a question, just state the guessed keyword with no other text except the keyword itself. "
                "DO NOT output any other text other than the guessed keyword. DO NOT refuse to guess. DO NOT REPEAT A PREVIOUS GUESS."
            ),
        }
        messages = [prompt, {
            "role": "user",
            "content": game_to_string(game) + "\nPrevious guesses:\n" + '\n'.join(guesses) + "\nYour guess: "
        }]
    elif role == "evaluator":
        prompt = {
            "role": "system",
            "content": (
                "You are an evaluator for the game 20 Questions. Given a keyword and a guess, return True if the guess was correct and False otherwise. "
                "The guess should refer to the same thing as the keyword but the guess should NOT be more vague or general than the keyword. DO NOT output anything except 'True' or 'False'."
                "Here are some guiding examples: \n\n"
                "Example 1:\nKeyword: USA\n Guess: United States\nEvaluation: True\n\n"
                "Example 2:\nKeyword: Peking duck\n Guess: duck\nEvaluation: False\n\n"
                "Example 3:\nKeyword: iPad\n Guess: tablet\nEvaluation: False\n\n"
                "Example 4:\nKeyword: pretoria south africa\n Guess: Pretoria\nEvaluation: True\n\n"
                "Example 5:\nKeyword: Bed and Breakfast\n Guess: BnB\nEvaluation: True\n\n"
                "Example 6:\nKeyword: Diet coke\n Guess: coca-cola\nEvaluation: False\n\n"
            ),
        }
        messages = [prompt, {
            "role": "user",
            "content": f"Keyword: {keyword}\nGuess: {guesses[-1]}\nEvaluation: "
        }]
    return messages


In [9]:
construct_messages([{"role": "questioner", "content": "Is it a place?"}], "answerer", "chair")

[{'role': 'system',
  'content': 'You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. Then the Guesser tries to guess the keyword based on the questions and answers in the game. The keyword is a specific thing, NOT a place and NOT a person. You are participating in a new game of 20 Questions. Your role is to be the Answerer. The keyword is chair. Answer only Yes or No based on the keyword. Do not output any other text.'},
 {'role': 'user',
  'content': "Answer the following question about the keyword: 'chair'. Is it a place?"}]

## Game Simulator

In [10]:
tree = things_tree

In [16]:
def simulate_game(keyword):
    print(f"USING MODEL: {MODEL}")
    print(f"KEYWORD: {keyword}")
    print()
    
    game = []

    questions = [step["content"] for step in game if step["role"] == "questioner"]
    answers = [step["content"] for step in game if step["role"] == "answerer"]
    guesses = []

    decision_node = tree
    
    for _ in range(20):
        if decision_node and decision_node.query:
            question = query_map[decision_node.query]
        else:
            question = completion(model=MODEL, messages=construct_messages(game, "questioner", keyword)).choices[0].message.content
    
        game.append({
            "role": "questioner",
            "content": question
        })

        print(f"questioner: {question}")

        if "OVER" in question:
            break
        
        questions.append(question)

        answer = completion(model=MODEL, messages=construct_messages(game, "answerer", keyword)).choices[0].message.content
    
        game.append({
            "role": "answerer",
            "content": answer
        })

        answers.append(answer)
        print(f"answerer: {answer}")

        if decision_node and decision_node.yes_branch != None and "yes" in answer.lower():
            decision_node = decision_node.yes_branch
        elif decision_node and decision_node.no_branch != None and "no" in answer.lower():
            decision_node = decision_node.no_branch
        else:
            decision_node = None
            
        guess = completion(model=MODEL, messages=construct_messages(game, "guesser", keyword, guesses)).choices[0].message.content

        guesses.append(guess)
        print(f"guesser: {guess}")

        evaluation = completion(model=EVALUATOR_MODEL, messages=construct_messages(game, "evaluator", keyword, guesses)).choices[0].message.content
        print(f"evaluator: {evaluation}")
        if "true" in evaluation.lower(): break

        print()

    return {
        "keyword": keyword,
        "questions": questions,
        "answers": answers,
        "guesses": guesses
    }

In [24]:
simulate_game("princess")

USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-ft-1:9tKOfrv9
KEYWORD: princess

questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: Computer
evaluator: False

questioner: Is it related to industrial production or manufacturing?
answerer: no
guesser: Tree
evaluator: False

questioner: Is it broadly related to entertainment or sports?
answerer: yes
guesser: Television
evaluator: False

questioner: Is it something a person can hold in their hand?
answerer: no
guesser: Theme Park
evaluator: False

questioner: Is it a type of equipment or gear used in sports?
answerer: no
guesser: Car
evaluator: False

questioner: Is it primarily used indoors?
answerer: no
guesser: Bicycle
evaluator: False

questioner: Is it a natural feature or phenomenon?
answerer: no
guesser: Stadium
evaluator: False

questioner: Is it something a person can ride on or in?
answerer: no
guesser: Roller coaster
evaluator: False

questioner: Is it related to a specific sport, such as footbal

{'keyword': 'princess',
 'questions': ['Is it related to food, beverages or cooking?',
  'Is it related to industrial production or manufacturing?',
  'Is it broadly related to entertainment or sports?',
  'Is it something a person can hold in their hand?',
  'Is it a type of equipment or gear used in sports?',
  'Is it primarily used indoors?',
  'Is it a natural feature or phenomenon?',
  'Is it something a person can ride on or in?',
  'Is it related to a specific sport, such as football or basketball?',
  'Is it a form of transportation used in recreational activities?',
  'Is it associated with a particular outdoor activity?',
  'Is it a type of game or play area?',
  'Is it an object specifically designed for entertainment purposes?',
  'Is it related to a specific type of entertainment, such as movies or music?',
  'Is it something that is primarily found in a movie theater?',
  'Is it associated with live performances or concerts?',
  'Is it a type of media format, such as a DV

## Generate all the games

In [25]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            record = json.loads(line)
            keyword = record.pop('keyword')
            result_dict[keyword] = record

    return result_dict

def read_json_to_dict(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

keyword_labels, results = read_jsonl_and_transform('20qs-data/keywords_data/labeled_keywords_v2.jsonl'), read_json_to_dict('20qs-data/keywords_data/labeling_results_v2.jsonl')

def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    return lines

keywords_list = read_file_to_list('20qs-data/keywords_data/keywords.txt')
similar_keywords_list = read_file_to_list('20qs-data/keywords_data/similar_keywords.txt')

keywords_list = keywords_list[1:]
print(f"Number of keywords in data: {len(keywords_list)}")

keyword_places = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'yes']
keyword_things = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'no']

print(f"Number of keywords labeled as things: {len(keyword_things)}.")

Number of keywords in data: 2046
Number of keywords labeled as things: 1347.


Read current games:

In [26]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Skip empty lines
                try:
                    record = json.loads(line)
                    keyword = record.pop('keyword')
                    result_dict[keyword] = record
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON for line: {line}")
                    print(f"Error message: {e}")

    return result_dict

# Path to the uploaded file
#file_path = '20qs-data/real_keyword_games_extended.jsonl'

# Read and transform the JSONL file
#result_dict = read_jsonl_and_transform(file_path)

# Get the current keywords with games
current_keywords_with_games = read_jsonl_and_transform('20qs-data/final_games/gpt-4o-200games-cleaned.jsonl')

In [27]:
game_results = []

In [33]:
import random
from tqdm import tqdm

dict_keywords = {d['keyword'] for d in game_results}

keywords_left = [keyword for keyword in keyword_things if (keyword not in dict_keywords) and (keyword not in current_keywords_with_games)]
similar_keywords_left = [keyword for keyword in similar_keywords_list if (keyword not in dict_keywords) and (keyword not in current_keywords_with_games)]

print(f"Keywords left: {len(keywords_left)}\n")
print(f"Similar Keywords left: {len(similar_keywords_left)}\n")

generated_keywords = random.choices(keywords_left, k=300)
generated_similar_keywords = random.choices(similar_keywords_left, k=300)

for generated_keyword in tqdm(generated_keywords+generated_similar_keywords, desc = 'games generated'):
    print("Generated keyword:", generated_keyword)
    game_result = simulate_game(generated_keyword)
    game_results.append(game_result)

Keywords left: 1208

Similar Keywords left: 1201



games generated:   0%|                                                                                                                                         | 0/600 [00:00<?, ?it/s]

Generated keyword: boa constrictor
USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-ft-1:9tKOfrv9
KEYWORD: boa constrictor

questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: laptop
evaluator: False

questioner: Is it related to industrial production or manufacturing?
answerer: no
guesser: Honey
evaluator: False

questioner: Is it broadly related to entertainment or sports?
answerer: no
guesser: Fringe
evaluator: False

questioner: Is it related to clothing, accessories or beauty products?
answerer: no
guesser: action figure
evaluator: False

questioner: Is it an animal?
answerer: yes
guesser: Cat
evaluator: False

questioner: Is it a domesticated animal?
answerer: no
guesser: elephant
evaluator: False

questioner: Is it a mammal?
answerer: no
guesser: Tiger
evaluator: False

questioner: Is it a type of bird?
answerer: no
guesser: Shark
evaluator: False

questioner: Is it a type of reptile?
answerer: yes
guesser: Crocodile
evaluator: False

questioner: I

games generated:   0%|▏                                                                                                                              | 1/600 [00:23<3:55:46, 23.62s/it]

evaluator: True
Generated keyword: exhibit sign
USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-ft-1:9tKOfrv9
KEYWORD: exhibit sign

questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: Coffee filter
evaluator: False

questioner: Is it related to industrial production or manufacturing?
answerer: no
guesser: Computer
evaluator: False

questioner: Is it broadly related to entertainment or sports?
answerer: no
guesser: Book
evaluator: False

questioner: Is it related to clothing, accessories or beauty products?
answerer: no
guesser: Chair
evaluator: False

questioner: Is it an animal?
answerer: no
guesser: Airplane
evaluator: False

questioner: Is it something a person can hold in their hand?
answerer: yes
guesser: Smartphone
evaluator: False

questioner: Is it related to electronics or technology?
answerer: no
guesser: Balloon
evaluator: False

questioner: Is it related to safety or safety equipment?
answerer: no
guesser: Key
evaluator: False

questioner: I

...

evaluator: False

Generated keyword: Paperweight
USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-ft-1:9tKOfrv9
KEYWORD: Paperweight

questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: Car
evaluator: False

questioner: Is it related to industrial production or manufacturing?
answerer: no
guesser: Computer
evaluator: False

questioner: Is it broadly related to entertainment or sports?
answerer: no
guesser: Tree
evaluator: False

questioner: Is it related to clothing, accessories or beauty products?
answerer: no
guesser: House
evaluator: False

questioner: Is it an animal?
answerer: no
guesser: Bicycle
evaluator: False

questioner: Is it something a person can hold in their hand?
answerer: yes
guesser: Book
evaluator: False

questioner: Is it related to electronics or technology?
answerer: no
guesser: Toy
evaluator: False

questioner: Is it related to safety or safety equipment?
answerer: no
guesser: Pen
evaluator: False

questioner: Is it commonly found i

games generated: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 599/600 [4:21:39<00:36, 36.83s/it]

evaluator: False

Generated keyword: Tent
USING MODEL: ft:gpt-4o-mini-2024-07-18:personal:20qs-ft-1:9tKOfrv9
KEYWORD: Tent

questioner: Is it related to food, beverages or cooking?
answerer: no
guesser: Car
evaluator: False

questioner: Is it related to industrial production or manufacturing?
answerer: no
guesser: Book
evaluator: False

questioner: Is it broadly related to entertainment or sports?
answerer: no
guesser: Smartphone
evaluator: False

questioner: Is it related to clothing, accessories or beauty products?
answerer: no
guesser: Mountain
evaluator: False

questioner: Is it an animal?
answerer: no
guesser: Tree
evaluator: False

questioner: Is it something a person can hold in their hand?
answerer: no
guesser: Frisbee
evaluator: False

questioner: Is it a living thing?
answerer: no
guesser: Airplane
evaluator: False

questioner: Is it furniture?
answerer: no
guesser: Balloon
evaluator: False

questioner: Is it broadly related to the arts or media?
answerer: no
guesser: Bridge


games generated: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [4:22:07<00:00, 26.21s/it]

evaluator: False



In [30]:
dict_keywords

set()

## Save as JSONL

In [34]:
with open("20qs-data/final_games/gpt-4o-mini-ft-600-games.jsonl", "w") as f:
    for i in game_results:
        json.dump(i, f)
        f.write('\n')